In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [2]:
import sys
import json

In [10]:
from pprint import pprint

In [37]:
sql = """
with temp as (
select 	a.id app_id,
		a.patient_id,
		a.doctor_id,
		a.finished at time zone 'UTC' cons_date,
		case
  		when a."source"='SCHEDULED' and specialty_id is not null then (
			select trim(name)
			from specialty s
			where s.id = a.specialty_id
		)
	    else (
			select trim(s.name)
			from specialty s
			inner join doctor_specialty ds
			   on ds.specialty_id = s.id
			inner join doctor_specialty_treatment dst
			   on dst.doctor_specialty_id = ds.id
			where dst.treatment_id = a.treatment_id
			limit 1 -- из-за одной криво заведенной записи.
		) end as specialty,
		p3.full_name doctor_provider,
		'CHAT' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_chat,
        'VIDEO' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_video,
        'AUDIO' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_audio,
        'PHONE' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_phone,
    		a.good,
		a.report_comment
--
from appointment a 
--
left join doctor on a.doctor_id = doctor.id 
left join person p_doctor on doctor.person_id = p_doctor.id 
left join provider p3 on p_doctor.provider_id = p3.id
--
where a.finished at time zone 'UTC' between '2019-07-01' and '2019-07-13'
and a.good
	      and not (
	      	a.patient_id in (
		      	select patient_id
		      	from patient_categories pcat
		      	inner join reference rf
		      	   on rf.id=pcat.reference_id
		      	where rf.code='TEST'
		      	)
		    or lower(a.report_comment)='тест'
	      	)  
and length(report_comment) >= 50
and ((lower(a.report_comment) not like 'к сожалению%') or ((lower(a.report_comment) like 'к сожалению%') and length(report_comment) >= 100))
and lower(a.report_comment) not like 'инициируйте%'
and lower(a.report_comment) not like 'тест%'
and lower(a.report_comment) not like '%ошибочн%'
and lower(a.report_comment) not like 'спасибо за%'
and lower(a.report_comment) not like '%тестова%'
)
--
select 	app_id,
		patient_id,
		doctor_id,
		cons_date,
		specialty,
		doctor_provider,
		report_comment
--
from temp 
--
where specialty in ('Педиатр', 'Терапевт')
and (is_chat or is_video or is_audio)



"""

In [38]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-07-13 15:18:57 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-07-13 15:18:57 creating ssh tunnel to 172.16.100.19 as root...
2020-07-13 15:18:57 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 42135
2020-07-13 15:18:57 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-13 15:18:57 @telemed query:
                    with temp as (
				    select 	a.id app_id,
				    		a.patient_id,
				    		a.doctor_id,
				    		a.finished at time zone 'UTC' cons_date,
				    		case
				      		when a."source"='SCHEDULED' and specialty_id is not null then (
				    			select trim(name)
				    			from specialty s
				    			where s.id = a.specialty_id
				    		)
				    	    else (
				    			select trim(s.name)
				    			from specialty s
				    			inner join doctor_specialty ds
				    			   on ds.s

In [39]:
data.cons_date = data.cons_date.astype(str)
data.cons_date = data.cons_date.str[:19]

In [40]:
data

,app_id,patient_id,doctor_id,cons_date,specialty,doctor_provider,report_comment
0,58400,1118726,502,2019-07-09 10:32:29,Терапевт,МК Доктор рядом,Рекомендовано:\nК-ЦИЯ ТЕРАПЕВТА\nузи БР.ПОЛОСТ...
1,57266,66694,3905,2019-07-01 13:03:44,Педиатр,Доктор рядом,Запишитесь к неврологу детскому. Очный осмотр ...
2,57636,1001231,4252,2019-07-03 16:27:38,Терапевт,МК Доктор рядом,В первую очередь необходимо исключать радикуло...
3,57673,1042727,2551,2019-07-03 20:03:53,Терапевт,МК Доктор рядом,Диагностическая гипотеза: последствия излеченн...
4,58374,807309,1739,2019-07-09 09:00:25,Терапевт,МК Доктор рядом,Диагностическая гипотеза: конъюнктивит на фоне...
...,...,...,...,...,...,...,...
353,58949,63901,4010,2019-07-12 22:24:53,Терапевт,МК Доктор рядом,"рекомендовано: консультация дерматолога, решен..."
354,58954,159734,4010,2019-07-12 23:38:41,Терапевт,МК Доктор рядом,отрубевидный лишай? дерматит?\nрекомендовано: ...
355,58876,1126661,502,2019-07-12 13:06:15,Терапевт,МК Доктор рядом,данных в наст момент неодостаточно.Необходимо ...
356,58939,1697765,2551,2019-07-12 20:43:30,Терапевт,МК Доктор рядом,Диагностическая гипотеза: поствакцинальная реа...


In [41]:
data.to_excel('DocPlus.xlsx', index=False)

In [42]:
result_file: str = sys.path[0] + '/docplus.json'
with open(result_file, 'w', encoding='utf-8') as f:
    json.dump(data.to_dict('records'), f, ensure_ascii=False)

In [43]:
with open(result_file, encoding='utf-8') as f:
    s = json.load(f)
pprint(s[0:2])

[{'app_id': 58400,
  'cons_date': '2019-07-09 10:32:29',
  'doctor_id': 502,
  'doctor_provider': 'МК Доктор рядом',
  'patient_id': 1118726,
  'report_comment': 'Рекомендовано:\n'
                    'К-ЦИЯ ТЕРАПЕВТА\n'
                    'узи БР.ПОЛОСТИ\n'
                    'биопрепараты и сорбенты продолжить \n'
                    'диета (рис, макароны, белые сухарики) \n'
                    '\n'
                    'с уважением,',
  'specialty': 'Терапевт'},
 {'app_id': 57266,
  'cons_date': '2019-07-01 13:03:44',
  'doctor_id': 3905,
  'doctor_provider': 'Доктор рядом',
  'patient_id': 66694,
  'report_comment': 'Запишитесь к неврологу детскому. Очный осмотр специалиста '
                    'необходим. Лечение корректирует доктор в динамике. \n'
                    'С ув Вам  доктор Мухтарова. Б.М.',
  'specialty': 'Педиатр'}]


In [33]:
sql2 = """
with temp as (
select 	a.id app_id,
		a.patient_id,
		a.doctor_id,
		a.finished at time zone 'UTC' cons_date,
		case
  		when a."source"='SCHEDULED' and specialty_id is not null then (
			select trim(name)
			from specialty s
			where s.id = a.specialty_id
		)
	    else (
			select trim(s.name)
			from specialty s
			inner join doctor_specialty ds
			   on ds.specialty_id = s.id
			inner join doctor_specialty_treatment dst
			   on dst.doctor_specialty_id = ds.id
			where dst.treatment_id = a.treatment_id
			limit 1 -- из-за одной криво заведенной записи.
		) end as specialty,
		p3.full_name doctor_provider,
		'CHAT' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_chat,
        'VIDEO' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_video,
        'AUDIO' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_audio,
        'PHONE' in (
            select reference.code
            from appointment_connection_type
    		left join reference on reference.id = appointment_connection_type.reference_id
    		where a.id = appointment_connection_type.appointment_id
    		) as is_phone,
    		a.good,
		a.report_comment
--
from appointment a 
--
left join doctor on a.doctor_id = doctor.id 
left join person p_doctor on doctor.person_id = p_doctor.id 
left join provider p3 on p_doctor.provider_id = p3.id
--
where a.finished at time zone 'UTC' between '2019-07-01' and '2019-07-13'
and a.good
	      and not (
	      	a.patient_id in (
		      	select patient_id
		      	from patient_categories pcat
		      	inner join reference rf
		      	   on rf.id=pcat.reference_id
		      	where rf.code='TEST'
		      	)
		    or lower(a.report_comment)='тест'
	      	)  
and length(report_comment) >= 50
and
(
 lower(a.report_comment)  like 'к сожалению%'
or lower(a.report_comment)  like 'инициируйте%'
or lower(a.report_comment)  like 'тест%'
or lower(a.report_comment)  like '%ошибочн%'
or lower(a.report_comment)  like 'спасибо за%'
or lower(a.report_comment)  like '%тестова%'
)
)
--
select 	app_id,
		patient_id,
		doctor_id,
		cons_date,
		specialty,
		doctor_provider,
		report_comment
--
from temp 
--
where specialty in ('Педиатр', 'Терапевт')
and (is_chat or is_video or is_audio)




"""

In [34]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data_error = connection.query(sql2)
connection.close()

2020-07-13 15:13:08 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-07-13 15:13:08 creating ssh tunnel to 172.16.100.19 as root...
2020-07-13 15:13:09 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 39711
2020-07-13 15:13:09 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-07-13 15:13:09 @telemed query:
                    with temp as (
				    select 	a.id app_id,
				    		a.patient_id,
				    		a.doctor_id,
				    		a.finished at time zone 'UTC' cons_date,
				    		case
				      		when a."source"='SCHEDULED' and specialty_id is not null then (
				    			select trim(name)
				    			from specialty s
				    			where s.id = a.specialty_id
				    		)
				    	    else (
				    			select trim(s.name)
				    			from specialty s
				    			inner join doctor_specialty ds
				    			   on ds.s

In [35]:
data_error.cons_date = data_error.cons_date.astype(str)
data_error.cons_date = data_error.cons_date.str[:19]

In [36]:
data_error.to_excel('DocPlus-error.xlsx', index=False)